In [ ]:
# Scorecard OpenLLMetry OpenAI Example

This notebook demonstrates how to use OpenLLMetry with OpenAI in Python to trace LLM calls and workflows in Scorecard.

In [ ]:
pip install traceloop-sdk openai

TRACELOOP_BASE_URL="https://telemetry.getscorecard.ai:4318" # Scorecard telemetry url
TRACELOOP_HEADERS="Authorization=Bearer <YOUR_SCORECARD_TELEMETRY_KEY>"

OPENAI_API_KEY=<YOUR_OPENAI_API_KEY>

In [ ]:
import os
from traceloop.sdk import Traceloop
from traceloop.sdk.decorators import workflow, task
from traceloop.sdk.instruments import Instruments
from openai import OpenAI

# Initialize OpenAI client
openai_client = OpenAI()

# Initialize Traceloop
Traceloop.init(disable_batch=True, instruments={Instruments.OPENAI})

In [ ]:
@task(name="joke_creation")
def create_joke():
    """Create a joke using OpenAI"""
    completion = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": "Tell me a joke"}]
    )
    
    return completion.choices[0].message.content


In [ ]:
@task(name="author_generation")
def generate_author(joke: str):
    """Generate an author for the given joke"""
    completion = openai_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": f"add a author to the joke:\n\n{joke}"}
        ]
    )
    
    return completion.choices[0].message.content


In [ ]:
@workflow(name="joke_generator")
def joke_workflow():
    """Main workflow that creates a joke and generates an author for it"""
    joke = create_joke()
    print(f"Generated joke: {joke}")
    
    joke_with_author = generate_author(joke)
    print(f"Joke with author: {joke_with_author}")
    
    return joke_with_author


In [ ]:
# Run the workflow
result = joke_workflow()
print("\nWorkflow completed!")


In [ ]:
## Viewing Traces in Scorecard

After running your application:

1. **Visit Scorecard**: Go to [app.getscorecard.ai](https://app.getscorecard.ai)
2. **Navigate to Traces**: Go to your project → Traces section

### What You'll See

- **Workflow spans**: High-level operations (`joke_generator`)
- **Task spans**: Individual operations (`joke_creation`, `author_generation`)  
- **LLM spans**: Automatic OpenAI API call instrumentation
- **Timing data**: Duration of each operation
- **Token usage**: Input/output tokens for LLM calls
- **Model information**: Which models were used

